In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

db_path = 'puzzles.db'
conn = sqlite3.connect(db_path)

In [ ]:
query = "SELECT length FROM solutions;"
df = pd.read_sql_query(query, conn)

length_counts = df['length'].value_counts().sort_index()

In [ ]:
plt.figure(figsize=(12, 6))

total = length_counts.sum()
length_percent = (length_counts / total) * 100

bars = plt.bar(length_percent.index, length_percent.values)  # type: ignore

plt.title(f'Распределение длин решений (всего {total} решений)', pad=20, fontsize=14)
plt.xlabel('Длина решений (шаги)', fontsize=12, labelpad=10)
plt.ylabel('Процент решений (%)', fontsize=12, labelpad=10)
plt.grid(axis='y', alpha=0.75)

for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.2f}%',
             ha='center', 
             va='bottom',
             fontsize=9)

plt.xticks(
    length_percent.index,
    fontsize=10
)

plt.yticks(fontsize=10)
plt.tight_layout()
plt.show()

In [ ]:
cursor = conn.cursor()

cursor.execute('''
    SELECT elo FROM puzzles WHERE elo != 1000
''')

data = cursor.fetchall()
data = [i[0] for i in data]

print('Average:', np.average(data))

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2,2)

ax = [ax1, ax2, ax3, ax4]
b = [10, 20, 40, 80]


for i in range(len(ax)):
    a = ax[i]
    bins = b[i]
    a.hist(data, bins=bins)
    a.set_title(f'По {bins} корзинам')

plt.suptitle('Распределение рейтингов задач')
fig.tight_layout()
fig.subplots_adjust(top=.88)

plt.show()

In [ ]:
cursor = conn.cursor()

cursor.execute('''
    SELECT 
        u.nickname,
        COUNT(p.userId) as play_count
    FROM users u
    LEFT JOIN played p ON p.userId = u.id
    GROUP BY u.id, u.nickname
    ORDER BY u.nickname;
''')

data = cursor.fetchall()

users = [i[0] for i in data]
puzzles = [i[1] for i in data]

plt.barh(users, puzzles)

plt.title('Число решенных задач')

plt.show()